# Gating of shading corrected fluoresnce
Use this notebook to select popluations of cells with a single fluorescent marker, so that they can be used in a subsequent demixing step

In [14]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib as mpl
import time
import inspect
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from matplotlib.widgets import LassoSelector
from matplotlib.path import Path as mplPath
import numpy as np
from pathlib import Path

from fluorescence_processing.fluor_gating_util import ScatterSelectorGating
import ipywidgets as widgets
from IPython.display import display
from bsccm import BSCCM

bsccm = BSCCM(str(Path.home()) + '/BSCCM_local/BSCCM/')
dataframe_saving_fullpath = str(Path.home()) + '/BSCCM_local/BSCCM/BSCCM_surface_markers.csv'
# For exporting figures
export_dir = '/home/henry/leukosight_data/figures/demixing/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
#Creat a figure for visualizing images and add a button for exporting
img_fig = plt.figure(figsize=(4,4))
export_text_box = widgets.Text(
                value='Export_name.pdf',
                placeholder='',
                description=''
            )

def export_fig(button):
    self.fig.savefig(export_dir + box.value, transparent=True)
    print('Saved to {}{}'.sformat(export_dir, box.value))

export_button = widgets.Button(description='Export figure')                
export_button.on_click(export_fig) 

display(widgets.HBox([export_text_box, export_button]))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
dimensions = {}
dimensions['antibodies'] = bsccm.index_dataframe['antibodies'].unique().tolist()
dimensions['batch'] = batches = bsccm.index_dataframe['batch'].unique().tolist()
dimensions['slide_replicate']  = bsccm.index_dataframe['slide_replicate'].unique().tolist()

# Setup top controls for selecting antobody/batch/slide
selector = None
def on_value_change(change):
    masks = [bsccm.index_dataframe[name] == widget_dict[name].value for name in widget_dict.keys()]
    total_mask = np.logical_and(np.logical_and(masks[0], masks[1]), masks[2])
    gui.set_selection(None)
    gui.change_data(total_mask)
        
widget_dict = {}
for name in dimensions.keys():
    widget_dict[name] = widgets.Dropdown(
        options=dimensions[name], value=dimensions[name][0],
        description=name + ':', disabled=False)
    widget_dict[name].observe(on_value_change, names='value')

    
channel_names = ['Fluor_426-446_shading_corrected', 
       'Fluor_500-550_shading_corrected', 
       'Fluor_550-570_shading_corrected', 
       'Fluor_585-625_shading_corrected', 
       'Fluor_627-673_shading_corrected', 
       'Fluor_690-_shading_corrected']

read_image_fn = lambda index: bsccm.read_image(index, contrast_type='dpc')


display(widgets.HBox(list(widget_dict.values())))

masks = [bsccm.index_dataframe[name] == widget_dict[name].value for name in widget_dict.keys()]
intial_mask = np.logical_and(np.logical_and(masks[0], masks[1]), masks[2])
gui = ScatterSelectorGating(bsccm.surface_marker_dataframe, intial_mask, channel_names, num_cols=1, 
                  export_dir=export_dir, read_image_fn=read_image_fn, img_fig=img_fig,
                           dataframe_saving_fullpath=dataframe_saving_fullpath)

# initialize
on_value_change(None)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(ToggleButtons(description='Plot_index', options=(), value=None), ToggleButtons(descripti…